In [2]:
import DataSetQuery

query = DataSetQuery.DataSetQuery('http://localhost:9000')

#Sets the output directory for the getnetcdffile call
query.setEnvironment('test','c:/earthwave/jonathan/testdata/')

print(query.getEnvironment())

print(query.getParentDataSets())

print(query.getDataSets('mtngla'))

print(query.getDataSetBoundingBox( 'mtngla','mtngla' ))

minX=-1900000
maxX=1100000
minY=-1100000
maxY=5600000
minT=1279356087000
maxT=1285386965000

gcs = query.getGridCells( 'mtngla','mtngla',minX,maxX,minY,maxY,minT,maxT ) 

shards = query.getShards('mtngla','mtngla',minX,maxX,minY,maxY,minT,maxT)

print(query.getNetCdfFile('mtngla','mtngla',minX,maxX,minY,maxY,minT,maxT))

{"name":"test","outputCdfPath":"c:/earthwave/jonathan/testdata/"}
{"dataSets":[{"name":"mtngla"},{"name":"scratch"}]}
{"dataSets":[{"name":"mtngla"}]}
{"gridCellMinX":-1900000,"gridCellMaxX":1100000,"gridCellMinY":-1100000,"gridCellMaxY":5600000,"minTime":1279356087000,"maxTime":1285386965000,"totalPoints":635961,"numberOfShards":256}
c:/earthwave/jonathan/testdata/data.nc


In [6]:
from pandas.io.json import json_normalize
import json


data = json.loads(gcs)
df = json_normalize(data['boxes'])

print(df[0:2])

totalPoints = df['totalPoints'].sum()
maxNumPoints = df['totalPoints'].max()

print('TP=[%d] MaxPoints in Cell=[%d]'%(totalPoints,maxNumPoints))   

dataShard = json.loads(shards)


dfShards = json_normalize(dataShard['shards'])

print(dfShards[0:2])



   gridCellMaxX  gridCellMaxY  gridCellMinX  gridCellMinY        maxTime  \
0       -300000        800000       -400000        700000  1284095031000   
1       -600000       5600000       -700000       5500000  1283795773000   

         minTime  numberOfShards  totalPoints  
0  1284095031000               1        32866  
1  1283795773000               1            3  
TP=[635961] MaxPoints in Cell=[104233]
            maxT          maxX           maxY           minT          minX  \
0  1285386965000  97288.981396  255996.974922  1285386965000  55565.774818   
1  1285386965000  56689.479345  569805.677418  1285386965000  55440.288649   

            minY  numberOfPoints  \
0  222051.218682               4   
1  568799.937460               2   

                                           shardName  
0  C:\Earthwave\/mtngla/swath/y2010/m9/cell_HMA_0...  
1  C:\Earthwave\/mtngla/swath/y2010/m9/cell_HMA_0...  
